In [1]:
# coding=utf-8
# author=yphacker

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import permutations

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
data = pd.concat([train_df, test_df], ignore_index=True)
data.drop('id', axis=1, inplace=True)
print(data.head())
print(data.taill())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   43    0   2       122   213    0        1      165      0      0.2      1   
1   66    0   2       146   278    0        0      152      0      0.0      1   
2   58    1   2       140   211    1        0      165      0      0.0      2   
3   63    0   0       124   197    0        1      136      1      0.0      1   
4   57    1   1       154   232    0        0      164      0      0.0      2   

   ca  thal  target  
0   0     2     1.0  
1   1     2     1.0  
2   0     2     1.0  
3   0     2     0.0  
4   1     2     0.0  
     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
298   44    0   2       118   242    0        1      149      0      0.3   
299   57    0   1       130   236    0        0      174      0      0.0   
300   59    1   2       150   212    1        1      157      0      1.6   
301   50    0   1       120   244    0        1      162      0      1.1   
3

In [3]:
data.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'serum_cholesterol', 'fasting_blood_sugar',
                'rest_ecg', 'max_heart_rate',
                'exercise_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

# x = data.corr()
# pd.DataFrame(x['target']).sort_values(by='target', ascending=False).style.background_gradient(cmap='copper')
data.sex = data.sex.map({0: 'female', 1: 'male'})

data.chest_pain_type = data.chest_pain_type.map(
    {1: 'angina pectoris', 2: 'atypical angina', 3: 'non-anginal pain', 4: 'SMI', 0: 'absent'})

data.fasting_blood_sugar = data.fasting_blood_sugar.map({0: 'lower than 120mg/ml', 1: 'greater than 120mg/ml'})

data.exercise_angina = data.exercise_angina.map({0: 'no', 1: 'yes'})

data.st_slope = data.st_slope.map({1: 'upsloping', 2: 'horizontal', 3: 'downsloping', 0: 'absent'})

data.thalassemia = data.thalassemia.map({1: 'normal', 2: 'fixed defect', 3: 'reversable defect', 0: 'absent'})

data.head()

# X = data.iloc[:, 0:13]
# Y = data.iloc[:, -1]

num_columns = ['age', 'resting_blood_pressure', 'serum_cholesterol', 'max_heart_rate', 'st_depression']

ss = StandardScaler()

# X[num_columns] = ss.fit_transform(X[num_columns])

# categorical_columns = X.select_dtypes(include=['object']).head().columns

# for column in categorical_columns:
#     dummies = pd.get_dummies(X[column], drop_first=True)
#     X[dummies.columns] = dummies
#     X.drop(column, axis=1, inplace=True)
#     print(X.shape)

data[num_columns] = ss.fit_transform(data[num_columns])

categorical_columns = data.select_dtypes(include=['object']).head().columns

for column in categorical_columns:
    dummies = pd.get_dummies(data[column], drop_first=True)
    data[dummies.columns] = dummies
    data.drop(column, axis=1, inplace=True)
    print(data.shape)

(303, 14)
(303, 16)
(303, 16)
(303, 16)
(303, 17)
(303, 19)


/Users/yphacker/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/yphacker/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [4]:
columns = data.columns.tolist()
columns.remove('target')

In [5]:
train_df = data[data['target'].notnull()]
X = train_df[columns]
Y = train_df['target']
# temp = X.copy()
# temp['target'] = Y

# d = temp.corr()
# pd.DataFrame(d['target']).sort_values(by='target', ascending=False).style.background_gradient(cmap='copper')

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = X, X, Y, Y

print("X-Train:", X_train.shape)
print("X-Test:", X_test.shape)
print("Y-Train:", y_train.shape)
print("Y-Test:", y_test.shape)

X-Train: (242, 18)
X-Test: (242, 18)
Y-Train: (242,)
Y-Test: (242,)


In [6]:
# def show_metrics(model):
#     fig = plt.figure(figsize=(25, 10))

#     # Confusion matrix
#     fig.add_subplot(121)
#     sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, annot_kws={"size": 16}, fmt='g')

#     # ROC Curve
#     fig.add_subplot(122)

#     auc_roc = roc_auc_score(y_test, model.predict(X_test))
#     fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

#     plt.plot(fpr, tpr, color='darkorange', lw=2, marker='o', label='Trained Model (area = {0:0.3f})'.format(auc_roc))
#     plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--', label='No Skill (area = 0.500)')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver operating characteristic')
#     plt.legend(loc="lower right")
#     plt.show()


# creating our model instance
log_reg = LogisticRegression(solver='lbfgs')

# fitting the model
log_reg.fit(X_train, y_train)

# predicting the target vectors
y_pred = log_reg.predict(X_test)

# show_metrics(log_reg)

accuracy = accuracy_score(y_pred, y_test)

print(f"The accuracy on test set using Logistic Regression is: {np.round(accuracy, 3) * 100.0}%")

print(classification_report(y_test, y_pred))

from sklearn.neighbors import KNeighborsClassifier

# creating a list of K's for performing KNN
my_list = list(range(0, 30))

# filtering out only the odd K values
neighbors = list(filter(lambda x: x % 2 != 0, my_list))

# list to hold the cv scores
cv_scores = []

# perform 10-fold cross validation with default weights
for k in neighbors:
    Knn = KNeighborsClassifier(n_neighbors=k, algorithm='brute')
    scores = cross_val_score(Knn, X_train, y_train, cv=10, scoring='accuracy', n_jobs=-1)
    cv_scores.append(scores.mean())

# finding the optimal k
optimal_k = neighbors[cv_scores.index(max(cv_scores))]
print("The optimal K value is with default weight parameter: ", optimal_k)

# # plotting accuracy vs K
# plt.plot(neighbors, cv_scores)
# plt.xlabel("Number of Neighbors K")
# plt.ylabel("Accuracy")
# plt.title("Accuracy vs K Plot for normal ")
# plt.grid()
# plt.show()

print("Accuracy scores for each K value is : ", np.round(cv_scores, 3))

from sklearn.metrics import accuracy_score

# create instance of classifier
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k, algorithm='kd_tree',
                                   n_jobs=-1)

# fit the model
knn_optimal.fit(X_train, y_train)

# predict on test vector
y_pred = knn_optimal.predict(X_test)

# evaluate accuracy score
accuracy = accuracy_score(y_test, y_pred) * 100
print(f"The accuracy on test set using KNN for optimal K = {optimal_k} is {np.round(accuracy, 3)}%")

# show_metrics(knn_optimal)

# Creating an instance of the classifier
svm = SVC(gamma='scale')

# training on train data
svm.fit(X_train, y_train)

# predicting on test data
y_pred = svm.predict(X_test)

# let's look at our accuracy
accuracy = accuracy_score(y_pred, y_test)

print(f"The accuracy on test set using SVC is: {np.round(accuracy, 3) * 100.0}%")

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

# First create the base model to tune
rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, cv=3, verbose=2,
                               random_state=42, n_jobs=-1)

# Fit the random search model
rf_random.fit(X_train, y_train)

rf_random.best_params_

# Creating an instance for the classifier
rf_best = RandomForestClassifier(**rf_random.best_params_)

# fitting the model
rf_best.fit(X_train, y_train)

# predict the labels
y_pred = rf_best.predict(X_test)

accuracy = accuracy_score(y_pred, y_test)

print(f"The accuracy on test set using RandomForest is: {np.round(accuracy, 3) * 100.0}%")

# creating a list of our models
ensembles = [log_reg, knn_optimal, rf_best, svm]

# Train each of the model
for estimator in ensembles:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

scores = [estimator.score(X_test, y_test) for estimator in ensembles]

scores

named_estimators = [
    ("log_reg", log_reg),
    ('random_forest', rf_best),
    ('svm', svm),
    ('knn', knn_optimal),
]

voting_clf = VotingClassifier(named_estimators)

voting_clf.fit(X_train, y_train)

# Let's look at our accuracy
acc = voting_clf.score(X_test, y_test)

print(f"The accuracy on test set using voting classifier is {np.round(acc, 3) * 100}%")

# to generate permutations of length three
perm = permutations(named_estimators, 3)

# to store the acc and classifiers
best_perm = []

# to store best classifier
best = []

# Traverse through the obtained permutations
for i in list(perm):
    # fit the classifier
    voting_clf = VotingClassifier(i)
    voting_clf.fit(X_train, y_train)

    # obtain accracy score and append it to the list
    acc = voting_clf.score(X_test, y_test)
    best_perm.append([acc, voting_clf])

# find out the maximum accuracy
maximum = max(best_perm, key=lambda x: x[0])

# there can be multiple permutations for which we get
# best score so find all of them and append to best
for i in range(len(best_perm)):
    if maximum[0] == best_perm[i][0]:
        best.append(best_perm[i][1])

acc_scores = []

for i in range(len(best)):
    voting_clf = best[i]
    # fit the classifier
    voting_clf.fit(X_train, y_train)
    # Let's look at our accuracy
    acc_scores.append(voting_clf.score(X_test, y_test))

print(f"The accuracy on test set using voting classifier is {np.round(max(acc_scores), 4) * 100}%")

The accuracy on test set using Logistic Regression is: 86.8%
              precision    recall  f1-score   support

         0.0       0.88      0.82      0.85       111
         1.0       0.86      0.91      0.88       131

   micro avg       0.87      0.87      0.87       242
   macro avg       0.87      0.86      0.87       242
weighted avg       0.87      0.87      0.87       242

The optimal K value is with default weight parameter:  17
Accuracy scores for each K value is :  [0.715 0.797 0.789 0.81  0.802 0.814 0.81  0.806 0.823 0.814 0.802 0.81
 0.814 0.814 0.81 ]
The accuracy on test set using KNN for optimal K = 17 is 83.884%
The accuracy on test set using SVC is: 90.10000000000001%
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 3 folds for each of 100 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.2min finished
/Users/yphacker/opt/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


The accuracy on test set using RandomForest is: 96.3%
Training the LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
Training the KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=17, p=2,
           weights='uniform')
Training the RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=1200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Training the SVC(C=1.0, cache_size=200, class_wei

In [7]:
X = data[data['target'].isnull()]
preds = voting_clf.predict(X[columns])

(61, 18)


/Users/yphacker/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
preds = rf_best.predict(X[columns])

In [10]:
submission = pd.DataFrame({'id': test_df['id'], 'target': preds})
submission.to_csv("../data/submission.csv", index=False, header=False)
submission.head()

,id,target
0,0,0.0
1,1,1.0
2,2,1.0
3,3,0.0
4,4,0.0
